In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import BorderlineSMOTE, SVMSMOTE, KMeansSMOTE, ADASYN
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter("ignore")
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll.stochastic import sample
import math
from sklearn.model_selection import GridSearchCV, ParameterGrid, train_test_split, cross_val_score
import re
import seaborn as sns
from scipy.stats import chi2_contingency
from subprocess import check_output
from joblib.logger import pprint
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

import lightgbm as lgb

%matplotlib inline

# 1.0 Data load

In [ ]:
all_df=pd.read_excel(r"Datasheet_2.xlsx")
all_df.shape

In [ ]:
print(all_df.shape)
all_df.head(1)

## Data creation

### Combinations possible

In [ ]:
all_df['New Ligand'].nunique()*all_df['New Substrate'].nunique()*all_df['CO Pressure (atm)'].nunique()*all_df['Solvent '].nunique()*all_df['Temperature (oC)'].nunique()

In [ ]:
print(all_df['New Ligand'].nunique(), all_df['New Substrate'].nunique(), all_df['CO Pressure (atm)'].nunique(), all_df['Solvent '].nunique(), all_df['Temperature (oC)'].nunique())

### Checking if values are at unique level

In [ ]:
all_df.columns

In [ ]:
duplicates = all_df.duplicated(subset=['New Ligand', 'BD 1-2', 'BD 4-5', 'BD 2-3',
       'BD 5-6', 'BD 2-7', 'BD 5-8', 'BD 7-8', 'A1-2-3', 'A4-5-6', 'A3-2-7',
       'A6-5-8', 'A1-2-7', 'A4-5-8', 'A2-7-8', 'A5-8-7', 'D1-2-7-8',
       'D4-5-8-7', 'D3-2-7-8', 'D6-5-8-7', 'D2-7-8-5', 'Non Bond 1-3',
       'Non Bond 4-6', 'Non Bond 3-6', 'Vib F 7-8', 'Vib F 2-3', 'Vib F 5-6',
       'Vib I 7-8', 'Vib I 2-3', 'Vib I 5-6', 'NMR 1', 'NMR 2', 'NMR 3',
       'NMR 4', 'NMR 5', 'NMR 6', 'NMR 7 ', 'NMR 8', 'q 1', 'q 2', 'q 3',
       'q 4', 'q 5', 'q 6', 'q 7 ', 'q 8', 'HOMO E', 'LUMO E', 'Area',
       'Polar SA', 'Volume', ' Ovality', 'Mol. Wt.', 'Dipole M', 'RCx', 'Rcy',
       'RCz', 'L1 (R1)', 'B1 (R1)', 'B5 (R1)', 'L1 (R2)', 'B1 (R2)', 'B5 (R2)'], keep='first')

num_duplicates = duplicates.sum()
num_non_duplicates = len(all_df) - num_duplicates

print(f"Number of duplicates: {num_duplicates}")
print(f"Number of non-duplicates: {num_non_duplicates}")


In [ ]:
all_df['New Ligand'].nunique()

In [ ]:
all_df[['New Ligand', 'BD 1-2', 'BD 4-5', 'BD 2-3',
       'BD 5-6', 'BD 2-7', 'BD 5-8', 'BD 7-8', 'A1-2-3', 'A4-5-6', 'A3-2-7',
       'A6-5-8', 'A1-2-7', 'A4-5-8', 'A2-7-8', 'A5-8-7', 'D1-2-7-8',
       'D4-5-8-7', 'D3-2-7-8', 'D6-5-8-7', 'D2-7-8-5', 'Non Bond 1-3',
       'Non Bond 4-6', 'Non Bond 3-6', 'Vib F 7-8', 'Vib F 2-3', 'Vib F 5-6',
       'Vib I 7-8', 'Vib I 2-3', 'Vib I 5-6', 'NMR 1', 'NMR 2', 'NMR 3',
       'NMR 4', 'NMR 5', 'NMR 6', 'NMR 7 ', 'NMR 8', 'q 1', 'q 2', 'q 3',
       'q 4', 'q 5', 'q 6', 'q 7 ', 'q 8', 'HOMO E', 'LUMO E', 'Area',
       'Polar SA', 'Volume', ' Ovality', 'Mol. Wt.', 'Dipole M', 'RCx', 'Rcy',
       'RCz', 'L1 (R1)', 'B1 (R1)', 'B5 (R1)', 'L1 (R2)', 'B1 (R2)', 'B5 (R2)']].drop_duplicates()

### Making dataset

In [ ]:
ligand_df = all_df[['New Ligand', 'BD 1-2', 'BD 4-5', 'BD 2-3',
       'BD 5-6', 'BD 2-7', 'BD 5-8', 'BD 7-8', 'A1-2-3', 'A4-5-6', 'A3-2-7',
       'A6-5-8', 'A1-2-7', 'A4-5-8', 'A2-7-8', 'A5-8-7', 'D1-2-7-8',
       'D4-5-8-7', 'D3-2-7-8', 'D6-5-8-7', 'D2-7-8-5', 'Non Bond 1-3',
       'Non Bond 4-6', 'Non Bond 3-6', 'Vib F 7-8', 'Vib F 2-3', 'Vib F 5-6',
       'Vib I 7-8', 'Vib I 2-3', 'Vib I 5-6', 'NMR 1', 'NMR 2', 'NMR 3',
       'NMR 4', 'NMR 5', 'NMR 6', 'NMR 7 ', 'NMR 8', 'q 1', 'q 2', 'q 3',
       'q 4', 'q 5', 'q 6', 'q 7 ', 'q 8', 'HOMO E', 'LUMO E', 'Area',
       'Polar SA', 'Volume', ' Ovality', 'Mol. Wt.', 'Dipole M', 'RCx', 'Rcy',
       'RCz', 'L1 (R1)', 'B1 (R1)', 'B5 (R1)', 'L1 (R2)', 'B1 (R2)', 'B5 (R2)']].drop_duplicates()

substrate_df = all_df[['New Substrate', 'S_BD 1-2', 'S_BD 3-4', 'S_Vib F 1-2', 'S_Vib F 3-4', 'S_Vib I 1-2',
       'S_Vib I 3-4', 'S_q 1', 'S_q 2', 'S_q 3', 'S_q 4', 'S_NMR 1', 'S_NMR 2',
       'S_NMR 3', 'S_NMR 4', 'S_HOMO E', 'S_LUMO E', 'S_Area', 'S_Polar SA',
       'S_Volume', 'S_ Ovality', 'S_Mol. Wt.', 'S_Dipole M', 'S_RCx', 'S_Rcy',
       'S_RCz', 'S_L1 (R1)', 'S_B1 (R1)', 'S_B5 (R1)', 'S_L1 (R2)',
       'S_B1 (R2)', 'S_B5 (R2)']].drop_duplicates()
substrate_df.sort_values('New Substrate')

In [ ]:
%%time

list1 = list(all_df['New Ligand'].unique())
list2 = list(all_df['New Substrate'].unique())
list3 = list(all_df['CO Pressure (atm)'].unique())
list4 = list(all_df['Solvent '].unique())
list5 = list(all_df['Temperature (oC)'].unique())

combinations = list(product(list1, list2, list3, list4, list5))

df = pd.DataFrame(combinations, columns=['New Ligand', 'New Substrate', 'CO Pressure (atm)', 'Solvent ', 'Temperature (oC)'])

df['Reaction ID (L-S-atm-Sol-Celcius)'] = df.apply(lambda row: f"{row['New Ligand']}-{row['New Substrate']}-{row['CO Pressure (atm)']}-{row['Solvent ']}-{row['Temperature (oC)']}", axis=1)

new_column_name = 'Reaction ID (L-S-atm-Sol-Celcius)'
df = df[[new_column_name] + [col for col in df.columns if col != new_column_name]]

df

In [ ]:
duplicates = df.duplicated(subset=['Reaction ID (L-S-atm-Sol-Celcius)', 'New Ligand', 'New Substrate',
       'CO Pressure (atm)', 'Solvent ', 'Temperature (oC)'], keep=False)
num_duplicates = duplicates.sum()
num_non_duplicates = len(df) - num_duplicates

print(f"Number of duplicates: {num_duplicates}")
print(f"Number of non-duplicates: {num_non_duplicates}")


In [ ]:
result = df.merge(ligand_df.drop_duplicates('New Ligand'), on='New Ligand', how='left')
result = result.merge(substrate_df.drop_duplicates('New Substrate'), on='New Substrate', how='left')
result.shape

In [ ]:
%%time
result.to_csv(r'reaction discovery/all_reactions_possible_dataset_v2.csv', index=False)

# Functions

In [ ]:
def data_preprocess(all_df):
    real_df = all_df[~all_df['Ligand'].isin(train_exclude)]
    oob_all_df=all_df[all_df['Ligand'].isin(oob_ligands)]
    print('Train Ligands:\n', real_df.Ligand.value_counts())
    print('---------------------------------------------------------------')
    print('OOB Ligands:\n', oob_all_df.Ligand.value_counts())
    print('---------------------------------------------------------------')
    real_df=real_df.iloc[:,3:]
    #print(real_df.head(1))
    return real_df, oob_all_df

In [ ]:
def smote_requirement(real_df, smote_required = True,smote=1):
    real_df['class']=np.where(real_df['Output (ee)%']>70,1,0)
    print('Real distribution (>70 is 1): \n', real_df['class'].value_counts())
    print('Real dataset: ', real_df.shape)
    minority_df=real_df[real_df['class']==0]
    X=real_df.iloc[:,:-1]
    y=real_df.iloc[:,-1]
    if smote_required == True:
        if  smote==1:
            sm = BorderlineSMOTE(random_state=2, kind = 'borderline-2')
            X_res, y_res = sm.fit_resample(X, y)

        elif smote==2:
            svm = SVMSMOTE(random_state=2)
            X_res, y_res = svm.fit_resample(X, y)
            
        elif smote==3:
            
            km = KMeansSMOTE(random_state=2)
            X_res, y_res = km.fit_resample(X, y)
        
        elif smote==4:
            ada = ADASYN(random_state=2)
            X_res, y_res = ada.fit_resample(X, y)
        
        print('SMOTE distribution (>70 is 1): \n', y_res.value_counts())
        print('SMOTE dataset: ', X_res.shape)
        X = X_res
        y = y_res
    else:
        pass

    return minority_df, X, y



In [ ]:
def data_split_scaling(X, random_state):
    X_org=X.iloc[:,:-1]
    y_org=X.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X_org, y_org, test_size=0.2, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def xgboost_model(X_train, X_test, y_train, y_test,
                  parameters_xgb, random_state, cv, early_stop , early_stop_rounds, reaction_df ):

    xgb1 = XGBRegressor(random_state=random_state)
    xgb_grid = GridSearchCV(xgb1,
                        parameters_xgb,
                        cv = cv,
                        n_jobs = -1,
                        verbose=True)
    if early_stop == True:
        xgb_grid.fit(X_train, y_train,  early_stopping_rounds=early_stop_rounds, eval_set=[(X_test, y_test)])
    else:
        xgb_grid.fit(X_train, y_train)
    print('Best model score: ', xgb_grid.best_score_)
    print('Best model parameters: ', xgb_grid.best_params_)

    prediction_train = xgb_grid.predict(X_train)
    # Predict on test data
    prediction = xgb_grid.predict(X_test)
    # Compute mean squared error
    mse_train = mean_squared_error(y_train, prediction_train, squared = False)
    mse_test = mean_squared_error(y_test, prediction, squared = False)

    print('Train RMSE: ', mse_train)
    print('Test RMSE: ', mse_test)
    

    prediction_oob = xgb_grid.predict(reaction_df)

    oob_df_predict = reaction_df.copy()
    oob_df_predict['prediction'] = prediction_oob
    
    return xgb_grid, mse_train, mse_test, xgb_grid.best_params_, oob_df_predict

# Modelling

## Data reading

In [ ]:
all_df=pd.read_excel(r"Datasheet.xlsx")
print(all_df.shape)
all_df.head()

In [ ]:
reaction_df=result.copy()
print(reaction_df.shape)
reaction_df.head(2)

In [ ]:
reaction_df.reset_index(inplace=True)
needed_columns = ['BD 1-2', 'BD 4-5', 'BD 2-3',
       'BD 5-6', 'BD 2-7', 'BD 5-8', 'BD 7-8', 'A1-2-3', 'A4-5-6', 'A3-2-7',
       'A6-5-8', 'A1-2-7', 'A4-5-8', 'A2-7-8', 'A5-8-7', 'D1-2-7-8',
       'D4-5-8-7', 'D3-2-7-8', 'D6-5-8-7', 'D2-7-8-5', 'Non Bond 1-3',
       'Non Bond 4-6', 'Non Bond 3-6', 'Vib F 7-8', 'Vib F 2-3', 'Vib F 5-6',
       'Vib I 7-8', 'Vib I 2-3', 'Vib I 5-6', 'NMR 1', 'NMR 2', 'NMR 3',
       'NMR 4', 'NMR 5', 'NMR 6', 'NMR 7 ', 'NMR 8', 'q 1', 'q 2', 'q 3',
       'q 4', 'q 5', 'q 6', 'q 7 ', 'q 8', 'HOMO E', 'LUMO E', 'Area',
       'Polar SA', 'Volume', ' Ovality', 'Mol. Wt.', 'Dipole M', 'RCx', 'Rcy',
       'RCz', 'L1 (R1)', 'B1 (R1)', 'B5 (R1)', 'L1 (R2)', 'B1 (R2)', 'B5 (R2)',
       'S_BD 1-2', 'S_BD 3-4', 'S_Vib F 1-2', 'S_Vib F 3-4', 'S_Vib I 1-2',
       'S_Vib I 3-4', 'S_q 1', 'S_q 2', 'S_q 3', 'S_q 4', 'S_NMR 1', 'S_NMR 2',
       'S_NMR 3', 'S_NMR 4', 'S_HOMO E', 'S_LUMO E', 'S_Area', 'S_Polar SA',
       'S_Volume', 'S_ Ovality', 'S_Mol. Wt.', 'S_Dipole M', 'S_RCx', 'S_Rcy',
       'S_RCz', 'S_L1 (R1)', 'S_B1 (R1)', 'S_B5 (R1)', 'S_L1 (R2)',
       'S_B1 (R2)', 'S_B5 (R2)', 'CO Pressure (atm)', 'Solvent ',
       'Temperature (oC)']
reaction_df = reaction_df[needed_columns]
all_df = all_df[needed_columns+['Output (ee)%']]

## Training model on all reactions (80:20)

### K means

#### XGB 

In [ ]:
"""
1: Borderline 2
2: SVM
3: Kmeans
4: Adasyn

"""
minority_df, X, y = smote_requirement(all_df, smote_required = True,smote=3)
X.head(1)

In [ ]:
X_train, X_test, y_train, y_test = data_split_scaling(X, random_state=660)

parameters_xgb = {'colsample_bytree': [0.7], 'gamma': [0.5], 'learning_rate': [0.03], 'max_depth': [6], 'min_child_weight': [4], 'n_estimators': [500], 'objective': ['reg:squarederror'], 'reg_alpha': [0.5], 'reg_lambda': [0.3], 'subsample': [0.3]}
xgb_grid, mse_train, mse_test, parameters, reaction_df_predicted = xgboost_model(X_train, X_test, y_train, y_test, parameters_xgb,
                                                                   random_state=660, cv = 5, early_stop = False ,early_stop_rounds = 5 , 
                                                                   reaction_df = reaction_df)